In [24]:
import torch
import numpy as np
from aihwkit.nn import AnalogLinear
from aihwkit.simulator.configs import InferenceRPUConfig
from aihwkit_test.customized_noise_pcm import TestNoiseModel  # 사용자 정의 noise model
from network import InfModel

# 테스트할 g_min, g_max 조합
g_minmax_list = [[0, 25], [1, 25], [5, 25], [10, 25]]

# 간단한 네트워크 (1-layer) 생성
model = AnalogLinear(10, 10)  # 10x10 weight matrix

for g_list in g_minmax_list:
    g_min, g_max = g_list
    print(f"\n=== Testing with g_min={g_min}, g_max={g_max} ===")

    # Inference Model 설정
    inf_model = InfModel(model=model, mode="cifar10", g_list=[g_min, g_max])
    analog_model = inf_model.ConvertModel()
    rpu_config = inf_model.SetConfig()

    # Conductance 변환
    for name, param in analog_model.named_parameters():
        # if "analog_module.shared_weights" in name:
        weights = param.data.cpu()
        
        # Convert to conductance
        conductance_pair = rpu_config.noise_model.g_converter.convert_to_conductances(weights)
        gp, gm = conductance_pair[0]  # (Gp, Gm)

        # `g_target` 값 설정 (다양한 conductance 값 테스트)
        g_target_values = [gp.min().item(), (gp.min().item() + gp.max().item()) / 2, gp.max().item()]

        for g_target in g_target_values:
            drift_coeff = rpu_config.noise_model.generate_drift_coefficients(torch.tensor([g_target]))
            
            print(f"g_target={g_target:.6f} → Drift Coeff: {drift_coeff.item():.6f}")




=== Testing with g_min=0, g_max=25 ===
g_target=25.000000 → Drift Coeff: 0.012782
g_target=25.000000 → Drift Coeff: 0.007258
g_target=25.000000 → Drift Coeff: 0.003650
g_target=0.000000 → Drift Coeff: 0.003835
g_target=12.194671 → Drift Coeff: 0.011689
g_target=24.389341 → Drift Coeff: 0.009997

=== Testing with g_min=1, g_max=25 ===
g_target=25.000000 → Drift Coeff: 0.012357
g_target=25.000000 → Drift Coeff: 0.007254
g_target=25.000000 → Drift Coeff: 0.004558
g_target=1.000000 → Drift Coeff: 0.015200
g_target=12.706884 → Drift Coeff: 0.012696
g_target=24.413769 → Drift Coeff: 0.025611

=== Testing with g_min=5, g_max=25 ===
g_target=25.000000 → Drift Coeff: 0.003815
g_target=25.000000 → Drift Coeff: 0.010204
g_target=25.000000 → Drift Coeff: 0.009440
g_target=5.000000 → Drift Coeff: 0.008534
g_target=14.755736 → Drift Coeff: 0.003544
g_target=24.511473 → Drift Coeff: 0.007166

=== Testing with g_min=10, g_max=25 ===
g_target=25.000000 → Drift Coeff: 0.020603
g_target=25.000000 → Drif

In [10]:
name, param = analog_model.named_parameters()

In [16]:
type(param[1])

torch.nn.parameter.Parameter

In [17]:
weights = param[1].data.cpu()

In [18]:
weights

tensor([ 0.1707, -0.2422,  0.1681,  0.0057,  0.2799,  0.2481,  0.1535,  0.0952,
         0.0088, -0.2817])